In [6]:
import sys
from dotenv import load_dotenv
import os

project_root = '/Users/ijunhyeong/Desktop/mcp/langgraph-mcp-agents/lee_mcp_tutorial'
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
    
    
os.chdir(project_root)

result = load_dotenv(override=True)
print(f"작업 디렉토리: {os.getcwd()}")
print(f".env 파일 로드: {'성공' if result else '실패'}")

작업 디렉토리: /Users/ijunhyeong/Desktop/mcp/langgraph-mcp-agents/lee_mcp_tutorial
.env 파일 로드: 성공


In [7]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import RunnableConfig
from utils import astream_graph


model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# 3 MCP server Integration
client = MultiServerMCPClient({
    "document-retriever": {
        "command": "/Users/ijunhyeong/Desktop/mcp/langgraph-mcp-agents/.venv/bin/python",
        "args": ["./mcp_server_rag.py"],
        "transport": "stdio",
    },
    "calculator": {
        "command": "/Users/ijunhyeong/Desktop/mcp/langgraph-mcp-agents/.venv/bin/python",
        "args": ["./mcp_server_calculator.py"],
        "transport": "stdio",
    },
    "code-executor": {
        "command": "/Users/ijunhyeong/Desktop/mcp/langgraph-mcp-agents/.venv/bin/python",
        "args": ["./mcp_server_code_executor.py"],
        "transport": "stdio",
    },
})

# MCP tool load
mcp_tools = await client.get_tools()

# TavilySearch load
tavily = TavilySearchResults(max_results=2, topic="news", days=2)

# tools integration
all_tools = mcp_tools + [tavily]

print(f"\n{'='*60}")
print(f"🎉 통합 완료! 사용 가능한 도구: {len(all_tools)}개")
print(f"{'='*60}\n")

for i, tool in enumerate(all_tools, 1):
    category = ""
    if tool.name == "retrieve":
        category = "📚 Document"
    elif tool.name in ["calculate", "percentage_change", "statistics_summary", "currency_convert", "compare_values"]:
        category = "🧮 Calculator"
    elif tool.name in ["execute_python", "create_visualization", "analyze_data", "read_csv", "save_to_csv"]:
        category = "💻 Code Executor"
    elif tool.name == "tavily_search_results_json":
        category = "🔍 Web Search"
        
    print(f"{i:2d}. {category:20s} {tool.name}")

print(f"\n{'='*60}\n")

# Create the agent with new API
agent = create_react_agent(
    model,
    all_tools,
    prompt="""You are a powerful AI agent with multiple capabilities:

    📚 Document Retrieval (retrieve):
       - Search information from iPhone technical documents

    🧮 Calculator (calculate, percentage_change, statistics_summary, currency_convert, compare_values):
       - Perform mathematical calculations and statistical analysis

    💻 Code Executor (execute_python, create_visualization, analyze_data, read_csv, save_to_csv):
       - Execute Python code for data analysis
       - Create visualizations (charts, graphs)
       - Process and save data files

    🔍 Web Search (tavily_search_results_json):
       - Search latest news and web information

    Choose the most appropriate tools based on the user's request.
    You can use multiple tools in sequence for complex tasks.
    Always explain which tools you're using and why.""",
    checkpointer=MemorySaver()
)

# 설정
config = RunnableConfig(recursion_limit=30, thread_id=1)

print("✅ Agent created successfully!")



🎉 통합 완료! 사용 가능한 도구: 12개

 1. 📚 Document           retrieve
 2. 🧮 Calculator         calculate
 3. 🧮 Calculator         percentage_change
 4. 🧮 Calculator         statistics_summary
 5. 🧮 Calculator         currency_convert
 6. 🧮 Calculator         compare_values
 7. 💻 Code Executor      execute_python
 8. 💻 Code Executor      create_visualization
 9. 💻 Code Executor      analyze_data
10. 💻 Code Executor      read_csv
11. 💻 Code Executor      save_to_csv
12. 🔍 Web Search         tavily_search_results_json


✅ Agent created successfully!


/var/folders/jj/9j_fmr5d7993hd80n8v4stn80000gn/T/ipykernel_54486/3962062005.py:62: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


## Test 1: simple visualization

In [3]:
await astream_graph(
    agent,
    {
        "messages": """
        Create a bar chart showing iPhone sales:
        - iPhone 17 Pro: 2.8 million
        - iPhone 17: 3.5 million
        - iPhone 16 Pro: 2.5 million
        - iPhone 16: 3.2 million

        Title: "iPhone Sales Comparison (Q3 2025)"
        Save as: sales_comparison.png
        """
    },
    config=config
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'll create a bar chart showing the iPhone sales data you provided. I'll use the `create_visualization` tool for this task.
🔧 Tool Selected: create_visualization

🔧 Tool Selected: 

🔧 Tool Selected: 

🔧 Tool Selected: 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

✅ Tool Executed: create_visualization
Visualization created successfully!
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Chart Type:     bar
Title:          iPhone Sales Comparison (Q3 2025)
Filename:       sales_comparison.png
Location:       /Users/ijunhyeong/Desktop/mcp/langgraph-mcp-agents/lee_mcp_tutorial/./output/sales_comparison.png
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I've created a bar chart with the iPhone sales data you provided. The chart shows:

- iPhone 17 Pro: 2.8 million units
- iPhone 17: 3.5 million units (highest sales)
- iPhone 16 Pro: 2.5 million units (lowest sales)
- iPhone

{'node': 'agent',
 'content': AIMessageChunk(content=[], additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None, 'model_provider': 'anthropic'}, id='lc_run--9f614a71-1fd3-4943-99c2-9c17201ecd4f', usage_metadata={'input_tokens': 3866, 'output_tokens': 195, 'total_tokens': 4061, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}, chunk_position='last'),
 'metadata': {'thread_id': 1,
  'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent',),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:36d1a6ac-4034-b46f-4ff6-dd11ced52913',
  'checkpoint_ns': 'agent:36d1a6ac-4034-b46f-4ff6-dd11ced52913',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000,
  'LANGSMITH_TRACING': 'true',
  'LANGSMITH_ENDPOINT': 'https://api.smith.langchain.com',
  'LANGSMITH_PROJECT': 'LangGraph-MCP-

## Test 2: Document Retrieve + Calculation

In [9]:
await astream_graph(
    agent,
    {
        "messages": """
        1. Use Document Retrieval to retrieve iPhone 17 Pro price from documents in euros.
        2. Use tavilysearch to check the currency between euro and Chinese RMB.
        2. Convert the euro price to RMB using the collected information.
        3. Show the result
        """
    },
    config=config
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'll help you convert the iPhone 17 Pro price from euros to Chinese RMB following your step-by-step instructions.

### Step 1: Retrieve iPhone 17 Pro price from documents in euros

First, let me search for the iPhone 17 Pro price in euros from the available documents.
🔧 Tool Selected: retrieve

🔧 Tool Selected: 

🔧 Tool Selected: 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

✅ Tool Executed: retrieve
iPhone 17 Pro Specifications 
 
Radio 
No 
USB 
USB Type-C 3.2 Gen 2, DisplayPort 
FEATURES 
Sensors 
Face ID, accelerometer, gyro, proximity, compass, barometer 
  
Ultra Wideband (UWB) support (gen2 chip) 
Emergency SOS, Messages and Find My via satellite 
BATTERY 
Type 
Li-Ion 3988 mAh - Nano SIM model 
Li-Ion 4252 mAh - eSIM only model 
Charging 
Wired, PD3.2, AVS, 50% in 20 min 
25W wireless MagSafe/Qi2, 50% in 30 min (15W - China) 
4.5W reverse wired 
MISC 
Colors 
Silver, Cosmic Orange, Deep 

{'node': 'agent',
 'content': AIMessageChunk(content=[], additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None, 'model_provider': 'anthropic'}, id='lc_run--2fbf9748-bfd8-4c54-ac11-ab3c77943113', usage_metadata={'input_tokens': 11042, 'output_tokens': 155, 'total_tokens': 11197, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}, chunk_position='last'),
 'metadata': {'thread_id': 1,
  'langgraph_step': 22,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent',),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:be852cd7-e302-0eea-956a-a8f790bc4b88',
  'checkpoint_ns': 'agent:be852cd7-e302-0eea-956a-a8f790bc4b88',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000,
  'LANGSMITH_TRACING': 'true',
  'LANGSMITH_ENDPOINT': 'https://api.smith.langchain.com',
  'LANGSMITH_PROJECT': 'LangGraph-M

In [12]:
await astream_graph(
    agent,
    {
        "messages": """
        Complete analysis task:

        Step 1: Use Document Retrieval to retrieve iPhone 17 and iPhone 16 battery capacity from documents
        Step 2: Calculate the percentage difference
        Step 3: Search for battery life comparison reviews on the web
        Step 4: Create a comparison chart showing:
           - Battery capacity (mAh)
           - Percentage difference
        Step 5: Save the data to battery_comparison.csv
        
        """
    },
    config=config
)



🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'll complete this battery analysis task step by step as requested.

### Step 1: Retrieve iPhone 17 and iPhone 16 battery capacity from documents

First, let me retrieve the iPhone 17 battery capacity:
🔧 Tool Selected: retrieve

🔧 Tool Selected: 

🔧 Tool Selected: 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

✅ Tool Executed: retrieve
iPhone 17 - Technical Specifications - Apple
https://www.apple.com/iphone-17/specs/?utm_source=chatgpt.com
7/15
iPhone 17 Pro Specifications 
 
Radio 
No 
USB 
USB Type-C 3.2 Gen 2, DisplayPort 
FEATURES 
Sensors 
Face ID, accelerometer, gyro, proximity, compass, barometer 
  
Ultra Wideband (UWB) support (gen2 chip) 
Emergency SOS, Messages and Find My via satellite 
BATTERY 
Type 
Li-Ion 3988 mAh - Nano SIM model 
Li-Ion 4252 mAh - eSIM only model 
Charging 
Wired, PD3.2, AVS, 50% in 20 min 
25W wireless MagSafe/Qi2, 50% in 30 min (15W - China) 
4.5W reverse wire

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'This request would exceed the rate limit for your organization (0862ef1d-8a76-46e3-88d6-4cc8c7a3a6b9) of 20,000 input tokens per minute. For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}, 'request_id': 'req_011CUL4FCMPHHQkLZ1EZZCgY'}